In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm import tqdm


In [2]:
train_dir = r"D:\Users\Lenova\Desktop\YukTha\dataset\train"
val_dir   = r"D:\Users\Lenova\Desktop\YukTha\dataset\val"


In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.RandomResizedCrop(224, scale=(0.7,1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])


In [4]:
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
val_data   = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=16)

class_names = train_data.classes
print(class_names)


['ai', 'real']


In [5]:
model = models.efficientnet_b0(weights="DEFAULT")


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Lenova/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [01:03<00:00, 335kB/s]


In [6]:
for param in model.features.parameters():
    param.requires_grad = False


In [7]:
num_features = model.classifier[1].in_features

model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 2)
)


In [8]:
device = torch.device("cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [9]:
epochs = 5
best_val_loss = float("inf")

for epoch in range(epochs):

    # TRAIN
    model.train()
    train_loss = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # VALIDATION
    model.eval()
    val_loss = 0
    correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()

    acc = correct / len(val_data)

    print(f"\nEpoch {epoch+1}")
    print("Train Loss:", train_loss/len(train_loader))
    print("Val Loss:", val_loss/len(val_loader))
    print("Val Acc:", acc)

    # SAVE BEST MODEL
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Model Saved!")


100%|██████████| 198/198 [06:10<00:00,  1.87s/it]



Epoch 1
Train Loss: 0.6494839635461268
Val Loss: 0.5810898299279966
Val Acc: 0.7716666666666666
Model Saved!


100%|██████████| 198/198 [06:28<00:00,  1.96s/it]



Epoch 2
Train Loss: 0.552580401301384
Val Loss: 0.4993886030034015
Val Acc: 0.8633333333333333
Model Saved!


100%|██████████| 198/198 [06:43<00:00,  2.04s/it]



Epoch 3
Train Loss: 0.4927139161813139
Val Loss: 0.4422496149414464
Val Acc: 0.8883333333333333
Model Saved!


100%|██████████| 198/198 [04:02<00:00,  1.22s/it]



Epoch 4
Train Loss: 0.446161072362553
Val Loss: 0.4041768414409537
Val Acc: 0.8883333333333333
Model Saved!


100%|██████████| 198/198 [03:53<00:00,  1.18s/it]



Epoch 5
Train Loss: 0.42288590275277993
Val Loss: 0.3765751993969867
Val Acc: 0.8916666666666667
Model Saved!


In [10]:
model.load_state_dict(torch.load("best_model.pth", map_location="cpu"))
model.eval()


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [11]:
prob = torch.softmax(output, dim=1)
confidence = prob.max().item()

if confidence < 0.75:
    print("Send to manual review")


NameError: name 'output' is not defined